<a href="https://colab.research.google.com/github/Jocelyn-git/Python_ML/blob/main/kaggle_Titanic_Machine_Learning_from_Disaster_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Random Forest

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Loading data
train_data = pd.read_csv("/content/drive/MyDrive/Colab-Notebooks/Titanic-Disaster/train.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Colab-Notebooks/Titanic-Disaster/test.csv")
test_ids = test_data["PassengerId"]

In [ ]:
#清除不重要的欄位，用平均數補空值
def clean(data):
    data = data.drop(["PassengerId", "Name", "Ticket", "Cabin", "Embarked"], axis=1)

    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].mean(), inplace=True)

    #data.Embarked.fillna("U", inplace=True)

    return data

In [ ]:
train_data = clean(train_data)
test_data = clean(test_data)

In [ ]:
train_data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [ ]:
#分割自變數與應變數，
#檢查data shape並欄位數值化
X = train_data.iloc[:, 1:]
Y = train_data.iloc[:, 0]
print(f"train data shape: {X.shape}")
print(f"test data shape: {test_data.shape}")

train data shape: (891, 6)
test data shape: (418, 6)


In [ ]:
dummy_x = pd.get_dummies(X)
print(dummy_x.shape)
dummy_x

(891, 7)


,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male
0,3,22.000000,1,0,7.2500,0,1
1,1,38.000000,1,0,71.2833,1,0
2,3,26.000000,0,0,7.9250,1,0
3,1,35.000000,1,0,53.1000,1,0
4,3,35.000000,0,0,8.0500,0,1
...,...,...,...,...,...,...,...
886,2,27.000000,0,0,13.0000,0,1
887,1,19.000000,0,0,30.0000,1,0
888,3,29.699118,1,2,23.4500,1,0
889,1,26.000000,0,0,30.0000,0,1


In [ ]:
#標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
sc_x = scaler.fit_transform(dummy_x)

In [ ]:
#RandomForest Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
#評估模型
depth = []
for i in tqdm(range(2,50,1)):
  rf = RandomForestClassifier(max_depth=i)
  scores = cross_val_score(rf, sc_x, Y, cv=5)
  depth.append(scores.mean())

100%|██████████| 48/48 [00:44<00:00,  1.07it/s]


In [ ]:
final_depth = np.argmax(np.array(depth)) + 2
print(final_depth)

10


In [ ]:
estimators = []
for i in tqdm(range(50,200,1)):
  rf = RandomForestClassifier(n_estimators=i, max_depth=final_depth)
  scores = cross_val_score(rf, sc_x, Y, cv=5)
  estimators.append(scores.mean())

100%|██████████| 150/150 [02:53<00:00,  1.15s/it]


In [ ]:
final_estimators = np.argmax(np.array(estimators)) + 50
print(final_estimators)

169


In [ ]:
final_rf = RandomForestClassifier(n_estimators=final_estimators, max_depth=final_depth)
final_rf.fit(sc_x, Y)

RandomForestClassifier(max_depth=10, n_estimators=169)

In [ ]:
#預測存活
dummy_test_data = pd.get_dummies(test_data)
sc_test_data_x = scaler.transform(dummy_test_data)
submission = final_rf.predict(sc_test_data_x)

In [ ]:
print(submission)

[0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 1 1 0 0 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 0 0 1 0 0 0 0 0 1 1 1 1 0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 0
 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 0 1 0 1 1 0 1 0 0 0]


In [ ]:
#Create Predit Data files
submission_data = pd.DataFrame({"PassengerId": test_ids.values, "Survived": submission })
submission_data.to_csv("submission_rf.csv", index=False)